# Proyecto B Etapa 2 Modelado, Simulación y Optimización

## Caso 1

In [2]:
import pandas as pd
import requests
import time

In [20]:
import pandas as pd
import requests
import time

# 1. Carga de datos
clients = pd.read_csv("data-caso1/clients.csv")
depots  = pd.read_csv("data-caso1/depots.csv")

# 2. Construcción de nodos con coordenadas
nodes = {0: (depots.loc[0, "Longitude"], depots.loc[0, "Latitude"])}
for _, row in clients.iterrows():
    nodes[int(row["ClientID"])] = (row["Longitude"], row["Latitude"])

# 3. Configuración de la API
api_key = '5b3ce3597851110001cf6248f4bace05742cd52a0f6b877ce5d52bffe229f05a5f3132ac87cef6d3'
headers = {
    'Authorization': api_key,
    'Content-Type': 'application/json'
}
url = 'https://api.openrouteservice.org/v2/directions/driving-car'

# 4. Creación de grafo con tuplas
grafo1 = {}

for i, coords_i in nodes.items():
    grafo1[i] = {}
    for j, coords_j in nodes.items():
        if i == j:
            # distancia y tiempo "infinitos" para mismo nodo
            grafo1[i][j] = (999.0, 999.0)
            continue

        params = {
            'start': f"{coords_i[0]},{coords_i[1]}",
            'end':   f"{coords_j[0]},{coords_j[1]}"
        }

        resp = requests.get(url, headers=headers, params=params)
        time.sleep(1.1)  # para no pasarte del límite

        if resp.status_code == 200:
            seg = resp.json()['features'][0]['properties']['segments'][0]
            dist = round(seg['distance'] / 1000, 2)  # km
            tiem = round(seg['duration'] / 60,   2)  # minutos

            # convertir ceros a 999.0
            if dist == 0.0: dist = 999.0
            if tiem == 0.0: tiem = 999.0

            grafo1[i][j] = (dist, tiem)
        else:
            # en caso de error HTTP
            grafo1[i][j] = (999.0, 999.0)
            print(f"Error {resp.status_code} entre {i} y {j}")

# 5. Mostrar resultado
import pprint
pprint.pprint(grafo1, width=60)



KeyboardInterrupt: 

In [4]:
import json

# Cargar el archivo JSON
with open('distancias-tiempo-api.json', 'r') as f:
    data = json.load(f)

# Crear la variable grafo
grafo1 = {}

for origen, destinos in data.items():
    grafo1[int(origen)] = {}
    for destino, valores in destinos.items():
        distancia = valores['distance_km']
        tiempo = valores['time_min']
        if distancia == 0.0:
            distancia = 999.0
        if tiempo == 0.0:
            tiempo = 999.0
        grafo1[int(origen)][int(destino)] = (distancia, tiempo)




In [21]:
print(grafo1)

{0: {0: (999.0, 999.0), 1: (27.19, 37.94)}}


In [6]:
from pyomo.environ import (
    ConcreteModel, Set, Param, Var, Binary, NonNegativeReals,
    Objective, Constraint, minimize, SolverFactory
)
import pandas as pd

# Datos 
clients = pd.read_csv("data-caso1/clients.csv")
vehicles = pd.read_csv("data-caso1/vehicles.csv")

clientes = clients["ClientID"].tolist()
vehiculos = vehicles["VehicleID"].tolist()
nodos = [0] + clientes

demanda = dict(zip(clients["ClientID"], clients["Demand"]))
capacidad = dict(zip(vehicles["VehicleID"], vehicles["Capacity"]))


distancias = {(i,j): grafo1[i][j][0] for i in nodos for j in nodos if i!=j}
tiempos    = {(i,j): grafo1[i][j][1] for i in nodos for j in nodos if i!=j}

# Construcción del modelo 
model = ConcreteModel()

model.N = Set(initialize=nodos)
model.C = Set(initialize=clientes)
model.V = Set(initialize=vehiculos)

model.d = Param(model.C, initialize=demanda) # demanda de los clientes
model.q = Param(model.V, initialize=capacidad) # capacidad de los vehículos
model.dist = Param(model.N, model.N, initialize=distancias, default=1e6)
model.t = Param(model.N, model.N, initialize=tiempos, default=0)

# Costos 
model.Pf = Param(initialize=15000)   # precio combustible por litro
model.Cm = Param(initialize=700)     # costo mantenimiento por km
model.Ce = Param(initialize=0)       # costo eléctrico
model.Ft = Param(initialize=5000)    # tarifa fija por km
model.Cv = Param(initialize=0)       # costo por viaje

# δ_v = 1 para todos los vehículos (todos camionetas)
delta = { v: 1 for v in vehiculos }
model.delta = Param(model.V, initialize=delta)

# Variables
model.x = Var(model.N, model.N, model.V, domain=Binary)
model.u = Var(model.C, model.V, domain=NonNegativeReals)

# Función Objetivo
def obj_custom(m):
    return sum((m.Pf * m.dist[i,j] * m.delta[v] + m.Cm + m.delta[v] * m.Ce + m.Ft + m.Cv * m.t[i,j]) * m.x[i,j,v] for i in m.N for j in m.N for v in m.V if i!=j)
model.obj = Objective(rule=obj_custom, sense=minimize)



# Restricciones
def cliente_una_vez(m, j):
    return sum(m.x[i,j,v] for i in m.N for v in m.V if i!=j) == 1

def cliente_salida(m, j):
    return sum(m.x[j,k,v] for k in m.N for v in m.V if j!=k) == 1

def flujo_cliente(m, j, v):
    return (sum(m.x[i,j,v] for i in m.N if i!=j) - sum(m.x[j,k,v] for k in m.N if k!=j)) == 0

def mtz_rule(m, i, j, v):
    if i!=j:
        return m.u[i,v] + m.d[j] - m.u[j,v] <= m.q[v]*(1 - m.x[i,j,v])
    return Constraint.Skip

def limites_u(m, i, v):
    return (m.d[i], m.u[i,v], m.q[v])

def capacidad_total(m, v):
    return sum(m.d[j]*m.x[i,j,v]
               for i in m.N for j in m.C if i!=j) <= m.q[v]


model.cliente_una_vez = Constraint(model.C, rule=cliente_una_vez)
model.cliente_salida = Constraint(model.C, rule=cliente_salida)
model.flujo = Constraint(model.C, model.V, rule=flujo_cliente)
model.salida_dep = Constraint(model.V, rule=lambda m,v: sum(m.x[0,j,v] for j in m.C) <= 1)
model.entrada_dep = Constraint(model.V, rule=lambda m,v: sum(m.x[i,0,v] for i in m.C) <= 1)
model.mtz = Constraint(model.C, model.C, model.V, rule=mtz_rule)
model.lim_u = Constraint(model.C, model.V, rule=limites_u)
model.cap_total = Constraint(model.V, rule=capacidad_total)

# Solución del modelo
solver = SolverFactory("glpk")
solver.options["tmlim"]  = 60     # segundos máximo
solver.options["mipgap"] = 0.05   # gap 5%
results = solver.solve(model, tee=True)

print("Status:     ", results.solver.status)
print("Termination:", results.solver.termination_condition)
print("Obj. value: ", model.obj())

# Print resultados
print("\n RUTAS ASIGNADAS POR VEHÍCULO:")
for v in model.V:
    rutas, actual, visitados = [], 0, set()
    while True:
        nxt = next(
            (j for j in model.N if j!=actual and model.x[actual,j,v].value>0.5),
            None
        )
        if nxt is None or nxt in visitados:
            break
        rutas.append((actual, nxt))
        visitados.add(nxt)
        actual = nxt
    print(f" Vehículo {v}: {rutas or 'no asignado'}")




GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --tmlim 60 --mipgap 0.05 --write /var/folders/zm/ycnc3kk55kl_sql6gtln4m_40000gn/T/tmp1r86ycj6.glpk.raw
 --wglp /var/folders/zm/ycnc3kk55kl_sql6gtln4m_40000gn/T/tmpomc357_k.glpk.glp
 --cpxlp /var/folders/zm/ycnc3kk55kl_sql6gtln4m_40000gn/T/tmpej5_be44.pyomo.lp
Reading problem data from '/var/folders/zm/ycnc3kk55kl_sql6gtln4m_40000gn/T/tmpej5_be44.pyomo.lp'...
/var/folders/zm/ycnc3kk55kl_sql6gtln4m_40000gn/T/tmpej5_be44.pyomo.lp:62050: warning: lower bound of variable 'x2' redefined
/var/folders/zm/ycnc3kk55kl_sql6gtln4m_40000gn/T/tmpej5_be44.pyomo.lp:62050: warning: upper bound of variable 'x2' redefined
5064 rows, 4992 columns, 37056 non-zeros
4800 integer variables, all of which are binary
66850 lines were read
Writing problem data to '/var/folders/zm/ycnc3kk55kl_sql6gtln4m_40000gn/T/tmpomc357_k.glpk.glp'...
56979 lines were written
GLPK Integer Optimizer 5.0
5064 rows, 4992 columns, 37056 non-zeros
4800 integ

KeyboardInterrupt: 

## Caso 2

In [7]:
import json
import pprint

# Cargar el archivo JSON 
with open('distancias-tiempo-api-carro-2.json', 'r') as f:
    data = json.load(f)

# Reconstruir grafo
grafo2 = {}
for origen_str, destinos in data.items():
    origen = int(origen_str)
    grafo2[origen] = {}
    for destino_str, valores in destinos.items():
        destino = int(destino_str)

        dist, tiem = valores

        if dist == 0.0: dist = 999.0
        if tiem == 0.0: tiem = 999.0
        grafo2[origen][destino] = (dist, tiem)




In [8]:
print(grafo2)

{0: {0: (999.0, 999.0), 1: (999.0, 999.0), 2: (999.0, 999.0), 3: (999.0, 999.0), 4: (999.0, 999.0), 5: (999.0, 999.0), 6: (999.0, 999.0), 7: (999.0, 999.0), 8: (999.0, 999.0), 9: (999.0, 999.0), 10: (999.0, 999.0), 11: (999.0, 999.0), 12: (999.0, 999.0), 13: (999.0, 999.0), 14: (999.0, 999.0)}, 1: {0: (999.0, 999.0), 1: (999.0, 999.0), 2: (14.73, 42.21), 3: (9.02, 24.12), 4: (4.3, 12.99), 5: (4.46, 13.49), 6: (999.0, 999.0), 7: (999.0, 999.0), 8: (11.43, 28.93), 9: (11.92, 33.63), 10: (1.58, 4.77), 11: (7.88, 22.33), 12: (999.0, 999.0), 13: (3.95, 11.94), 14: (999.0, 999.0)}, 2: {0: (999.0, 999.0), 1: (14.73, 42.21), 2: (999.0, 999.0), 3: (9.18, 25.02), 4: (10.43, 29.22), 5: (13.35, 37.97), 6: (999.0, 999.0), 7: (999.0, 999.0), 8: (11.59, 29.84), 9: (2.83, 8.63), 10: (16.1, 46.28), 11: (7.81, 22.77), 12: (999.0, 999.0), 13: (13.76, 39.33), 14: (999.0, 999.0)}, 3: {0: (999.0, 999.0), 1: (9.02, 24.12), 2: (9.18, 25.02), 3: (999.0, 999.0), 4: (4.72, 11.13), 5: (7.64, 19.88), 6: (999.0, 99

In [9]:
import math
import pandas as pd

# Función para calcular la distancia entre dos puntos geográficos usando longitud y latitud
def haversine(coord1, coord2):
    lon1, lat1 = coord1
    lon2, lat2 = coord2
    
    R = 6371.0  # Radio de la Tierra en km
    
    # Grados a radianes
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    
    # Fórmula matemática
    a = math.sin(delta_phi/2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    distance = R * c
    return distance

# Cargar datos de clientes y depósito
clients = pd.read_csv("data-caso2/clients.csv")
depots = pd.read_csv("data-caso2/depots.csv")

# Crear un diccionario de nodos con coordenadas
nodes = {}

# Agregar el depósito como nodo central
for _, row in depots.iterrows():
    nodes[row["LocationID"]] = (row["Longitude"], row["Latitude"])

# Agregar los clientes como nodos
for _, row in clients.iterrows():
    nodes[row["LocationID"]] = (row["Longitude"], row["Latitude"])

# Cargar datos de vehículos
vehicles = pd.read_csv("data-caso2/vehicles.csv")

# Filtrar solo los vehículos cuyo Type sea 'drone'
drones = vehicles[vehicles["Type"] == "drone"]

# Crear un diccionario para almacenar las velocidades de los drones
drone_speeds = dict(zip(drones["VehicleID"], drones["Speed"]))

# Diccionario para almacenar los grafos por dron
graphs = {}

for drone_id, speed in drone_speeds.items():
    # Convertir la velocidad de km/h a km/min
    speed_km_min = float(speed) / 60.0 if speed != "N/A" else None
    
    # Crear un grafo para cada dron
    graphs[drone_id] = {}
    
    for i, coord_i in nodes.items():
        graphs[drone_id][i] = {}  # Inicializar el nodo en el grafo
        for j, coord_j in nodes.items():
            if i == j:
                # Si son el mismo nodo, asignar valores grandes
                graphs[drone_id][i][j] = (999, 999)  # Distancia y tiempo grandes
            else:
                # Calcular la distancia usando la fórmula de Haversine
                distance_km = haversine(coord_i, coord_j)
                if speed_km_min:
                    # Calcular el tiempo en minutos
                    duration_min = distance_km / speed_km_min
                    graphs[drone_id][i][j] = (round(distance_km, 3), round(duration_min, 3))
                else:
                    # Si no hay velocidad, asignar valores grandes
                    graphs[drone_id][i][j] = (999, 999)

# Ejemplo de uso
dron_id = 3
nodo_origen = 1
nodo_destino = 12

if dron_id in graphs:
    if nodo_destino in graphs[dron_id][nodo_origen]:
        dist, tiempo = graphs[dron_id][nodo_origen][nodo_destino]
        print(f"Dron {dron_id} de nodo {nodo_origen} a nodo {nodo_destino}:")
        print(f"  Distancia = {dist} km, Tiempo = {tiempo} min")
    else:
        print(f"No hay datos para el dron {dron_id} entre nodo {nodo_origen} y nodo {nodo_destino}.")
else:
    print(f"No hay datos para el dron {dron_id}.")

Dron 3 de nodo 1 a nodo 12:
  Distancia = 2.396 km, Tiempo = 1.065 min


In [11]:
import math
import pandas as pd

# Función para calcular la distancia entre dos puntos geográficos usando longitud y latitud
def haversine(coord1, coord2):
    lon1, lat1 = coord1
    lon2, lat2 = coord2
    
    R = 6371.0  # Radio de la Tierra en km
    
    # Grados a radianes
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    
    # Fórmula matemática
    a = math.sin(delta_phi/2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    distance = R * c
    return distance

# Cargar datos de clientes y depósito
clients = pd.read_csv("data-caso2/clients.csv")
depots = pd.read_csv("data-caso2/depots.csv")

# Crear un diccionario de nodos con coordenadas
nodes = {}

# Agregar el depósito como nodo central
for _, row in depots.iterrows():
    nodes[row["LocationID"]] = (row["Longitude"], row["Latitude"])

# Agregar los clientes como nodos
for _, row in clients.iterrows():
    nodes[row["LocationID"]] = (row["Longitude"], row["Latitude"])

# Cargar datos de vehículos
vehicles = pd.read_csv("data-caso2/vehicles.csv")

# Filtrar solo los vehículos cuyo Type sea 'drone'
drones = vehicles[vehicles["Type"] == "drone"]

# Crear un diccionario para almacenar las velocidades de los drones
drone_speeds = dict(zip(drones["VehicleID"], drones["Speed"]))

# Diccionario para almacenar el grafo de distancias y tiempos para drones
graphdron = {}

for i, coord_i in nodes.items():
    graphdron[i] = {}  # Inicializar el nodo en el grafo
    for j, coord_j in nodes.items():
        if i == j:
            # Si son el mismo nodo, asignar valores grandes
            graphdron[i][j] = (999, 999)  # Distancia y tiempo grandes
        else:
            # Calcular la distancia usando la fórmula de Haversine
            distance_km = haversine(coord_i, coord_j)
            # Calcular el tiempo promedio para drones (usando la velocidad promedio de los drones)
            if len(drone_speeds) > 0:
                avg_speed_km_min = sum(float(speed) for speed in drone_speeds.values() if speed != "N/A") / (60.0 * len(drone_speeds))
                duration_min = distance_km / avg_speed_km_min
                graphdron[i][j] = (round(distance_km, 3), round(duration_min, 3))
            else:
                # Si no hay velocidad disponible, asignar valores grandes
                graphdron[i][j] = (999, 999)

# Ejemplo de uso
nodo_origen = 0
nodo_destino = 12

if nodo_origen in graphdron and nodo_destino in graphdron[nodo_origen]:
    dist, tiempo = graphdron[nodo_origen][nodo_destino]
    print(f"De nodo {nodo_origen} a nodo {nodo_destino}:")
    print(f"  Distancia = {dist} km, Tiempo = {tiempo} min")
else:
    print(f"No hay datos entre nodo {nodo_origen} y nodo {nodo_destino}.")

De nodo 0 a nodo 12:
  Distancia = 1.279 km, Tiempo = 0.569 min


Sin restriccion de ventana

In [81]:
from pyomo.environ import (
    ConcreteModel, Set, Param, Var, Binary, NonNegativeReals,
    Objective, Constraint, minimize, SolverFactory
)
import pandas as pd

# Cargar datos
clients = pd.read_csv("data-caso2/clients.csv")
vehicles = pd.read_csv("data-caso2/vehicles.csv")

clientes = clients["ClientID"].tolist()
vehiculos = vehicles["VehicleID"].tolist()
nodos = [0] + clientes  # Nodo 0 es el depósito

# Diccionarios
demanda = dict(zip(clients["ClientID"], clients["Demand"]))
capacidad = dict(zip(vehicles["VehicleID"], vehicles["Capacity"]))

# Asignar tipo de vehículo: 1 = 4x4, 0 = dron
tipo_vehiculo = dict(zip(vehicles["VehicleID"], (1 if t.lower() == "4x4" else 0 for t in vehicles["Type"])))

# Construir matrices de distancia y tiempo
distancias, tiempos = {}, {}
for i in nodos:
    for j in nodos:
        if i != j:
            for v in vehiculos:
                try:
                    if tipo_vehiculo[v] == 1:  # Camioneta
                        dist, tiempo_viaje, *_ = grafo2[i][j]
                    else:  # Dron
                        dist, tiempo_viaje, *_ = graphdron[i][j]
                except KeyError:
                    dist, tiempo_viaje = 999, 0
                distancias[(i, j, v)] = dist
                tiempos[(i, j, v)] = tiempo_viaje

# Modelo
model = ConcreteModel()

model.N = Set(initialize=nodos)
model.C = Set(initialize=clientes)
model.V = Set(initialize=vehiculos)

model.d = Param(model.C, initialize=demanda)
model.q = Param(model.V, initialize=capacidad)
model.delta = Param(model.V, initialize=tipo_vehiculo)

# Costos
model.Pf = Param(initialize=15000)
model.Cm = Param(initialize=700)
model.Ce = Param(initialize=0)
model.Ft = Param(initialize=5000)
model.Cv = Param(initialize=0)

# Datos de red
model.dist = Param(model.N, model.N, model.V, initialize=distancias, default=1e6)
model.t = Param(model.N, model.N, model.V, initialize=tiempos, default=0)

# Variables
model.x = Var(model.N, model.N, model.V, domain=Binary)  # Ruta (i,j) con vehículo v
model.u = Var(model.C, model.V, domain=NonNegativeReals)  # Carga acumulada en nodo i con vehículo v

# Objetivo
def obj_cost(m):
    return sum(
        (m.Pf * m.dist[i, j, v] * m.delta[v] + m.Cm + m.delta[v]*m.Ce + m.Ft + m.Cv * m.t[i, j, v]) * m.x[i, j, v]
        for i in m.N for j in m.N for v in m.V if i != j
    )
model.obj = Objective(rule=obj_cost, sense=minimize)

# Restricciones

# Cada cliente es visitado exactamente una vez
model.cliente_una_vez = Constraint(
    model.C,
    rule=lambda m, j: sum(m.x[i, j, v] for i in m.N for v in m.V if i != j) == 1
)

# Cada cliente tiene una salida
model.cliente_salida = Constraint(
    model.C,
    rule=lambda m, j: sum(m.x[j, k, v] for k in m.N for v in m.V if j != k) == 1
)

# Flujo para cada vehículo
model.flujo = Constraint(
    model.C, model.V,
    rule=lambda m, j, v: sum(m.x[i, j, v] for i in m.N if i != j) - sum(m.x[j, k, v] for k in m.N if k != j) == 0
)

# Salida y entrada del depósito
model.salida_dep = Constraint(
    model.V,
    rule=lambda m, v: sum(m.x[0, j, v] for j in m.C) <= 1
)
model.entrada_dep = Constraint(
    model.V,
    rule=lambda m, v: sum(m.x[i, 0, v] for i in m.C) <= 1
)

# Restricción de capacidad (MTZ para subtours)
# Restricción de subtour con control de carga (MTZ reforzado)
def mtz_rule(m, i, j, v):
    if i != j:
        return m.u[i, v] + m.d[j] - m.u[j, v] <= m.q[v] * (1 - m.x[i, j, v]) + m.q[v] * (1 - m.x[i, j, v])
    else:
        return Constraint.Skip
model.mtz = Constraint(model.C, model.C, model.V, rule=mtz_rule)

# Límite inferior y superior de u
model.lim_u = Constraint(
    model.C, model.V,
    rule=lambda m, i, v: (m.d[i], m.u[i, v], m.q[v])
)

# Capacidad total por vehículo
model.cap_total = Constraint(
    model.V,
    rule=lambda m, v: sum(m.d[j] * m.x[i, j, v] for i in m.N for j in m.C if i != j) <= m.q[v]
)



# Resolver
solver = SolverFactory("glpk")
solver.options["tmlim"] = 30
solver.options["mipgap"] = 0.05
results = solver.solve(model, tee=True)

# Resultados
print("Status:     ", results.solver.status)
print("Termination:", results.solver.termination_condition)
print("Obj. value: ", model.obj())

# Rutas, tiempos totales y demanda total
print("\nRUTAS ASIGNADAS POR VEHÍCULO:")
for v in model.V:
    rutas, actual, visitados = [], 0, set()
    tiempo_total = 0  # Variable para acumular el tiempo total del vehículo
    demanda_total = 0  # Variable para acumular la demanda total del camino
    while True:
        nxt = next((j for j in model.N if j != actual and model.x[actual, j, v].value > 0.5), None)
        if nxt is None or nxt in visitados:
            break
        rutas.append((actual, nxt))
        tiempo_total += model.t[actual, nxt, v]  # Sumar el tiempo de viaje
        if nxt != 0:  # Excluir el nodo 0 (depósito) de la demanda
            demanda_total += demanda[nxt]
        visitados.add(nxt)
        actual = nxt
    print(f" Vehículo {v}: {rutas or 'no asignado'}")
    print(f" Tiempo total del vehículo {v}: {tiempo_total:.2f} minutos")
    print(f" Demanda total del camino (sin incluir el nodo 0): {demanda_total}")

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --tmlim 30 --mipgap 0.05 --write /var/folders/zm/ycnc3kk55kl_sql6gtln4m_40000gn/T/tmpwjforbmd.glpk.raw
 --wglp /var/folders/zm/ycnc3kk55kl_sql6gtln4m_40000gn/T/tmpe8kmaq1d.glpk.glp
 --cpxlp /var/folders/zm/ycnc3kk55kl_sql6gtln4m_40000gn/T/tmp7_0nna1m.pyomo.lp
Reading problem data from '/var/folders/zm/ycnc3kk55kl_sql6gtln4m_40000gn/T/tmp7_0nna1m.pyomo.lp'...
/var/folders/zm/ycnc3kk55kl_sql6gtln4m_40000gn/T/tmp7_0nna1m.pyomo.lp:8177: warning: lower bound of variable 'x2' redefined
/var/folders/zm/ycnc3kk55kl_sql6gtln4m_40000gn/T/tmp7_0nna1m.pyomo.lp:8177: warning: upper bound of variable 'x2' redefined
707 rows, 672 columns, 4744 non-zeros
630 integer variables, all of which are binary
8807 lines were read
Writing problem data to '/var/folders/zm/ycnc3kk55kl_sql6gtln4m_40000gn/T/tmpe8kmaq1d.glpk.glp'...
7463 lines were written
GLPK Integer Optimizer 5.0
707 rows, 672 columns, 4744 non-zeros
630 integer variables

CON RESTRICCION DE VENTANA

In [16]:
from pyomo.environ import (
    ConcreteModel, Set, Param, Var, Binary, NonNegativeReals,
    Objective, Constraint, minimize, SolverFactory
)
import pandas as pd

# Cargar datos
clients = pd.read_csv("data-caso2/clients.csv")
vehicles = pd.read_csv("data-caso2/vehicles.csv")

clientes = clients["ClientID"].tolist()
vehiculos = vehicles["VehicleID"].tolist()
nodos = [0] + clientes  # Nodo 0 es el depósito

# Diccionarios
demanda = dict(zip(clients["ClientID"], clients["Demand"]))
capacidad = dict(zip(vehicles["VehicleID"], vehicles["Capacity"]))

# Asignar tipo de vehículo: 1 = 4x4, 0 = dron
tipo_vehiculo = dict(zip(vehicles["VehicleID"], (1 if t.lower() == "4x4" else 0 for t in vehicles["Type"])))

# Procesar ventanas de tiempo con ajuste
def parse_time_window(time_window):
    start, end = time_window.split('-')
    h1, m1 = map(int, start.split(':'))
    h2, m2 = map(int, end.split(':'))
    # Convertir a minutos y ajustar los límites
    start_minutes = max(0, h1 * 60 + m1 - 60)  # Restar 1 hora al límite inferior
    end_minutes = h2 * 60 + m2 + 60            # Añadir 1 hora al límite superior
    return start_minutes, end_minutes


time_windows = {row.ClientID: parse_time_window(row.TimeWindow) for _, row in clients.iterrows()}

# Construir matrices de distancia y tiempo
distancias, tiempos = {}, {}
for i in nodos:
    for j in nodos:
        if i != j:
            for v in vehiculos:
                try:
                    if tipo_vehiculo[v] == 1:  # Camioneta
                        dist, tiempo_viaje, *_ = grafo2[i][j]
                    else:  # Dron
                        dist, tiempo_viaje, *_ = graphdron[i][j]
                except KeyError:
                    dist, tiempo_viaje = 999, 0
                distancias[(i, j, v)] = dist
                tiempos[(i, j, v)] = tiempo_viaje

# Modelo
model = ConcreteModel()

model.N = Set(initialize=nodos)
model.C = Set(initialize=clientes)
model.V = Set(initialize=vehiculos)

model.d = Param(model.C, initialize=demanda)
model.q = Param(model.V, initialize=capacidad)
model.delta = Param(model.V, initialize=tipo_vehiculo)

# Costos
model.Pf = Param(initialize=15000)
model.Cm = Param(initialize=700)
model.Ce = Param(initialize=0)
model.Ft = Param(initialize=5000)
model.Cv = Param(initialize=0)

# Datos de red
model.dist = Param(model.N, model.N, model.V, initialize=distancias, default=1e6)
model.t = Param(model.N, model.N, model.V, initialize=tiempos, default=0)

# Variables
model.x = Var(model.N, model.N, model.V, domain=Binary)  # Ruta (i,j) con vehículo v
model.u = Var(model.C, model.V, domain=NonNegativeReals)  # Carga acumulada en nodo i con vehículo v
model.t_arrival = Var(model.C, model.V, domain=NonNegativeReals)  # Tiempo de llegada a cada cliente

# Objetivo
def obj_cost(m):
    return sum(
        (m.Pf * m.dist[i, j, v] * m.delta[v] + m.Cm + m.delta[v]*m.Ce + m.Ft + m.Cv * m.t[i, j, v]) * m.x[i, j, v]
        for i in m.N for j in m.N for v in m.V if i != j
    )
model.obj = Objective(rule=obj_cost, sense=minimize)

# Restricciones

# Cada cliente es visitado exactamente una vez
model.cliente_una_vez = Constraint(
    model.C,
    rule=lambda m, j: sum(m.x[i, j, v] for i in m.N for v in m.V if i != j) == 1
)

# Cada cliente tiene una salida
model.cliente_salida = Constraint(
    model.C,
    rule=lambda m, j: sum(m.x[j, k, v] for k in m.N for v in m.V if j != k) == 1
)

# Flujo para cada vehículo
model.flujo = Constraint(
    model.C, model.V,
    rule=lambda m, j, v: sum(m.x[i, j, v] for i in m.N if i != j) - sum(m.x[j, k, v] for k in m.N if k != j) == 0
)

# Salida y entrada del depósito
model.salida_dep = Constraint(
    model.V,
    rule=lambda m, v: sum(m.x[0, j, v] for j in m.C) <= 1
)
model.entrada_dep = Constraint(
    model.V,
    rule=lambda m, v: sum(m.x[i, 0, v] for i in m.C) <= 1
)

# Restricción de capacidad (MTZ para subtours)
def mtz_rule(m, i, j, v):
    if i != j:
        return m.u[i, v] + m.d[j] - m.u[j, v] <= m.q[v] * (1 - m.x[i, j, v])
    else:
        return Constraint.Skip
model.mtz = Constraint(model.C, model.C, model.V, rule=mtz_rule)

# Límite inferior y superior de u
model.lim_u = Constraint(
    model.C, model.V,
    rule=lambda m, i, v: (m.d[i], m.u[i, v], m.q[v])
)

# Capacidad total por vehículo
model.cap_total = Constraint(
    model.V,
    rule=lambda m, v: sum(m.d[j] * m.x[i, j, v] for i in m.N for j in m.C if i != j) <= m.q[v]
)

# Restricción de ventanas de tiempo
def time_window_lower_rule(m, j, v):
    return m.t_arrival[j, v] >= time_windows[j][0]
model.time_window_lower = Constraint(model.C, model.V, rule=time_window_lower_rule)

def time_window_upper_rule(m, j, v):
    return m.t_arrival[j, v] <= time_windows[j][1]
model.time_window_upper = Constraint(model.C, model.V, rule=time_window_upper_rule)

# Restricción para calcular el tiempo de llegada acumulativo
def arrival_time_rule(m, i, j, v):
    if i != j and i in m.C and j in m.C:
        return m.t_arrival[j, v] >= m.t_arrival[i, v] + m.t[i, j, v] - (1 - m.x[i, j, v]) * 1e6
    return Constraint.Skip
model.arrival_time = Constraint(model.C, model.C, model.V, rule=arrival_time_rule)

# Resolver
solver = SolverFactory("glpk")
solver.options["tmlim"] = 30
solver.options["mipgap"] = 0.05
results = solver.solve(model, tee=True)

# Resultados
print("Status:     ", results.solver.status)
print("Termination:", results.solver.termination_condition)
print("Obj. value: ", model.obj())

# Rutas, tiempos totales y demanda total
print("\nRUTAS ASIGNADAS POR VEHÍCULO:")
for v in model.V:
    rutas, actual, visitados = [], 0, set()
    tiempo_total = 0  # Variable para acumular el tiempo total del vehículo
    demanda_total = 0  # Variable para acumular la demanda total del camino
    while True:
        nxt = next((j for j in model.N if j != actual and model.x[actual, j, v].value > 0.5), None)
        if nxt is None or nxt in visitados:
            break
        rutas.append((actual, nxt))
        tiempo_total += model.t[actual, nxt, v]  # Sumar el tiempo de viaje
        if nxt != 0:  # Excluir el nodo 0 (depósito) de la demanda
            demanda_total += demanda[nxt]
        visitados.add(nxt)
        actual = nxt
    print(f" Vehículo {v}: {rutas or 'no asignado'}")
    print(f" Tiempo total del vehículo {v}: {tiempo_total:.2f} minutos")
    print(f" Demanda total del camino (sin incluir el nodo 0): {demanda_total}")

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --tmlim 30 --mipgap 0.05 --write /var/folders/zm/ycnc3kk55kl_sql6gtln4m_40000gn/T/tmpnnb1xcjj.glpk.raw
 --wglp /var/folders/zm/ycnc3kk55kl_sql6gtln4m_40000gn/T/tmp3x9n_6pd.glpk.glp
 --cpxlp /var/folders/zm/ycnc3kk55kl_sql6gtln4m_40000gn/T/tmpou5rcfby.pyomo.lp
Reading problem data from '/var/folders/zm/ycnc3kk55kl_sql6gtln4m_40000gn/T/tmpou5rcfby.pyomo.lp'...
/var/folders/zm/ycnc3kk55kl_sql6gtln4m_40000gn/T/tmpou5rcfby.pyomo.lp:11831: warning: lower bound of variable 'x2' redefined
/var/folders/zm/ycnc3kk55kl_sql6gtln4m_40000gn/T/tmpou5rcfby.pyomo.lp:11831: warning: upper bound of variable 'x2' redefined
1337 rows, 714 columns, 6466 non-zeros
630 integer variables, all of which are binary
12461 lines were read
Writing problem data to '/var/folders/zm/ycnc3kk55kl_sql6gtln4m_40000gn/T/tmp3x9n_6pd.glpk.glp'...
10529 lines were written
GLPK Integer Optimizer 5.0
1337 rows, 714 columns, 6466 non-zeros
630 integer var

## Caso 3
